In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException, StaleElementReferenceException
from selenium import webdriver

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import random
import re

In [2]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1120, 1000)

In [12]:
def get_reviews(company_name):
    url = 'https://www.glassdoor.com/Reviews/index.htm'
    driver.get(url)
    while True:
        try:
            search_input = driver.find_element(By.XPATH, "//input[@placeholder='Search for a Company']")
            search_button = driver.find_element(By.XPATH, "//button[@aria-label='Search']")
            search_input.send_keys(company_name)
            search_button.click()
            break
        except NoSuchElementException:
            driver.get(url)

    time.sleep(1)
    # always click the first result link
    try:
        company_link = driver.find_elements(By.XPATH, "//div[@id='Discover']//a[@data-test='company-tile']")[0]
        company_link.click()
    except (NoSuchElementException, IndexError) as error:
        return None, None
    
    time.sleep(1)
    # may pop up survy window
    try:
        close_btn = driver.find_element(By.XPATH, "//div[@class='qual_x_close']")
        close_btn.click()
    except NoSuchElementException:
        pass
    
    trial_count = 0
    while trial_count < 10:
        try:
            reviews_link = driver.find_element(By.XPATH, "//a[@data-label='Reviews']")
            reviews_link.click()
        except (NoSuchElementException, StaleElementReferenceException) as error:
            trial_count += 1


    time.sleep(1)

    need_crawl = True
    while need_crawl:
        try:
            see_more_button = driver.find_element(By.XPATH, "//button[@data-test='highlights-toggle-link']")
            see_more_button.click()
            pros = driver.find_elements(By.XPATH, "//li[@data-term-type='PRO']")
            review_pros = []
            for pro in pros:
                review_pros.append(pro.text.strip())
            # print(review_pros)
            cons = driver.find_elements(By.XPATH, "//li[@data-term-type='CON']")
            review_cons = []
            for con in cons:
                review_cons.append(con.text.strip())
            need_crawl = False
            return review_pros, review_cons
            # print(review_cons)
        except (NoSuchElementException, StaleElementReferenceException) as error:
            try:
                close_btn = driver.find_element(By.XPATH, "//div[@class='qual_x_close']")
                close_btn.click()
            except NoSuchElementException:
                need_crawl = False
                pass
    return None, None

In [13]:
with open('data/companies.csv','r') as fh:
    company_list = []
    lines = fh.readlines()
    for line in lines[1:]:
        company_list.append(line.split(',')[1])

all_reviews_list = []
for company in company_list[100:200]:
    pros, cons = get_reviews(company)
    review_dict = {'name': company, 'Pros': pros, 'Cons': cons}
    all_reviews_list.append(review_dict)

In [15]:
all_reviews_list_4 = []
for company in company_list[300:400]:
    pros, cons = get_reviews(company)
    review_dict = {'name': company, 'Pros': pros, 'Cons': cons}
    all_reviews_list_4.append(review_dict)

In [16]:
pd.DataFrame(all_reviews_list_4).to_csv('./glassdoor_reviews/reviews_300_399.csv')